In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import label_binarize

df = pd.read_csv(r'./cleaned_up_part2.csv')


def create_windowed_dataset(data, target, window=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(data) - window, step):
        v = data.iloc[i : (i + window)].values
        labels = target.iloc[i : i + window]
        Xs.append(v)
        ys.append(labels.mode()[0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)




In [16]:

data = df.drop(columns=["label"])
target = df["label"]
X_train, y_train = create_windowed_dataset(data, target, window=1000, step=1)
y_train = label_binarize(y_train, classes=['walking', 'falling', 'kneeling'])
X_train.shape, y_train.shape

((16499, 1000, 6), (16499, 3))

In [21]:
# import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.build(input_shape=(None, X_train.shape[1], X_train.shape[2]))
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 256)              138240    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                                 
Total params: 171,523
Trainable params: 171,523
Non-trainable params: 0
_________________________________________________________________


In [20]:
# tf callbacks
SavingBestmodel = keras.callbacks.ModelCheckpoint(
    "best_model.h5", monitor="val_loss", mode="min", verbose=1, save_best_only=True
)



In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=128,
    validation_split = 0.2,
    shuffle=True,
    callbacks=[SavingBestmodel]
)

## Loading model
- load & split data into window
- split into train and test data
- one hot encoding of ytrain, xtest
- predicting and calculating accuracy

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X, y = create_windowed_dataset(data, target, window=1000, step=1)
xtrain, xtest, ytrain, ytest =  train_test_split(X, y, shuffle = True)
ytrain = label_binarize(ytrain, classes=['walking', 'falling', 'kneeling'])
ytest = label_binarize(ytest, classes=['walking', 'falling', 'kneeling'])

best_model = tf.keras.models.load_model('best_model.h5')
predictions = best_model.predict(xtrain)
test_prediction = best_model.predict(xtest)

print("train-acc:", accuracy_score(ytrain.argmax(axis=1), predictions.argmax(axis=1)))
print("test-acc:", accuracy_score(ytest.argmax(axis=1), test_prediction.argmax(axis=1)))


## Predicting one sample
- load a sample
- expands to batch
- predict with model

In [3]:
import tensorflow as tf
best_model = tf.keras.models.load_model('best_model.h5')


In [13]:
sample = df.iloc[15000:16000].drop(columns = ['label'])
true_label = df.iloc[15000:16000]['label'].mode()[0]
sample.values
sample_batch = np.expand_dims(sample.values, axis = 0)
print("shape", sample_batch.shape)
classes=['walking', 'falling', 'kneeling']

shape (1, 1000, 6)


In [14]:
classes[best_model.predict(sample_batch).argmax(axis=1)[0]]

1/1 [==============================] - 0s 39ms/step


'falling'

## Converting to tflite
- save whole model in folder
- convert to tflite
- save to .tflite format

In [8]:
import tensorflow as tf
model=tf.keras.models.load_model('best_model.h5')
model.save('LSTM_model')


In [9]:
import tensorflow as tf


converter = tf.lite.TFLiteConverter.from_saved_model('/content/LSTM_model')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)



706488

## Notes
in case if tflite model didn't fit the device
- use less LSTM units <128>
- less dense units <128>
- remove differntiation-part

## Quantization

In [21]:
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 1000, 6)
      yield [data.astype(np.float32)]

In [25]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model('/content/LSTM_model')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter._experimental_lower_tensor_list_ops = True  # Disable lower tensor list ops
tflite_quant_model = converter.convert()


In [26]:
open("quantized_model.tflite", "wb").write(tflite_quant_model)

193032

In [28]:
!xxd -i quantized_model.tflite > quantized_model_data.cc